In [0]:
%sql
USE CATALOG catalog_cp;
CREATE SCHEMA IF NOT EXISTS silver;
USE silver;

CREATE TABLE IF NOT EXISTS region (
    regionkey INT NOT NULL,
    name STRING NOT NULL,
    comment STRING,
    PRIMARY KEY (regionkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS nation (
    nationkey INT NOT NULL,
    name STRING NOT NULL,
    regionkey INT NOT NULL,
    comment STRING,
    PRIMARY KEY (nationkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS part (
    partkey INT NOT NULL,
    name STRING NOT NULL,
    mfgr STRING,
    brand STRING,
    type STRING,
    size INT,
    container STRING,
    retailprice DECIMAL(12,2),
    comment STRING,
    PRIMARY KEY (partkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS supplier (
    suppkey INT NOT NULL,
    name STRING NOT NULL,
    address STRING,
    nationkey INT NOT NULL,
    phone STRING,
    acctbal DECIMAL(12,2),
    comment STRING,
    PRIMARY KEY (suppkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS partsupp (
    partkey INT NOT NULL,
    suppkey INT NOT NULL,
    availqty INT,
    supplycost DECIMAL(12,2),
    comment STRING,
    PRIMARY KEY (partkey, suppkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS customer (
    custkey INT NOT NULL,
    name STRING NOT NULL,
    address STRING,
    nationkey INT NOT NULL,
    phone STRING,
    acctbal DECIMAL(12,2),
    mktsegment STRING,
    comment STRING,
    PRIMARY KEY (custkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS orders (
    orderkey INT NOT NULL,
    custkey INT NOT NULL,
    orderstatus STRING,
    totalprice DECIMAL(12,2),
    orderdate DATE,
    orderpriority STRING,
    clerk STRING,
    shippriority INT,
    comment STRING,
    PRIMARY KEY (orderkey)
) USING DELTA;

CREATE TABLE IF NOT EXISTS lineitem (
    orderkey INT NOT NULL,
    partkey INT NOT NULL,
    suppkey INT NOT NULL,
    linenumber INT NOT NULL,
    quantity DECIMAL(12,2),
    extendedprice DECIMAL(12,2),
    discount DECIMAL(12,2),
    tax DECIMAL(12,2),
    returnflag STRING,
    linestatus STRING,
    shipdate DATE,
    commitdate DATE,
    receiptdate DATE,
    shipinstruct STRING,
    shipmode STRING,
    comment STRING,
    PRIMARY KEY (orderkey, linenumber)
) USING DELTA;


In [0]:
%sql
USE CATALOG catalog_cp;
USE SCHEMA silver;

MERGE INTO region AS target
USING bronze.region AS source
ON target.r_regionkey = source.r_regionkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO nation AS target
USING bronze.nation AS source
ON target.n_nationkey = source.n_nationkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO part AS target
USING bronze.part AS source
ON target.p_partkey = source.p_partkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO supplier AS target
USING bronze.supplier AS source
ON target.s_suppkey = source.s_suppkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO partsupp AS target
USING bronze.partsupp AS source
ON target.ps_partkey = source.ps_partkey AND target.ps_suppkey = source.ps_suppkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO customer AS target
USING bronze.customer AS source
ON target.c_custkey = source.c_custkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO orders AS target
USING bronze.orders AS source
ON target.o_orderkey = source.o_orderkey
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO lineitem AS target
USING bronze.lineitem AS source
ON target.l_orderkey = source.l_orderkey AND target.l_linenumber = source.l_linenumber
WHEN NOT MATCHED THEN INSERT *;

In [0]:
from pyspark.sql.functions import sha2, concat_ws, col
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

tables = ['customer', 'orders', 'lineitem', 'nation', 'part', 'partsupp', 'supplier', 'region']

source_catalog = "samples"
source_schema = "tpch"
target_catalog = "catalog_cp"
target_schema = "silver"

def get_row_hashes_and_count(catalog, schema, table):
    df = spark.table(f"{catalog}.{schema}.{table}")
    row_hashes = df.select(sha2(concat_ws("||", *df.columns), 256).alias("row_hash"))
    return row_hashes, df.count()

all_match = True

for table in tables:
    source_hashes, source_count = get_row_hashes_and_count(source_catalog, source_schema, table)
    target_hashes, target_count = get_row_hashes_and_count(target_catalog, target_schema, table)

    row_count_match = (source_count == target_count)
    missing_in_target = source_hashes.subtract(target_hashes)
    missing_in_source = target_hashes.subtract(source_hashes)

    hash_match = (missing_in_target.count() == 0 and missing_in_source.count() == 0)

    if row_count_match and hash_match:
        print(f"Table '{table}': MATCH (rows: {source_count})")
    else:
        print(f"Table '{table}': MISMATCH")
        if not row_count_match:
            print(f"   - Row count differs: source={source_count}, target={target_count}")
        if missing_in_target.count() > 0:
            print(f"   - {missing_in_target.count()} rows missing in target")
        if missing_in_source.count() > 0:
            print(f"   - {missing_in_source.count()} rows missing in source")
        all_match = False

if all_match:
    print("\nAll tables match.")
else:
    print("\n One or more tables do not match.")
